In [8]:
import pandas as pd
import polars as pl
from datetime import datetime, timedelta

In [6]:
hoy = datetime.today()
lunes = hoy - timedelta(days=hoy.weekday())
viernes = lunes + timedelta(days=4)
fecha_min = lunes.strftime("%d/%m")
fecha_max = viernes.strftime("%d/%m")
fecha_max

'19/12'

In [11]:
data_euroleague = (
    pl.read_csv(
        "https://raw.githubusercontent.com/IvoVillanueva/EUROLEAGUE-BOXSCORE/refs/heads/main/data/euroleague_boxscore_2025_26.csv"
    )
    .with_columns(
        pl.col("date").str.to_datetime("%Y-%m-%d %H:%M:%S").alias("date"),
        
        pl.col("minutes").str.replace("DNP", "0:00").alias("minutes")
    )
    .with_columns(
        pl.when(pl.col("team_name") == "FC BARCELONA")
            .then(pl.lit("Barça"))
            .when(pl.col("team_name") == "VALENCIA BASKET")
            .then(pl.lit("Valencia Basket"))
            .when(pl.col("team_name") == "REAL MADRID")
            .then(pl.lit("Real Madrid"))
            .when(pl.col("team_name") == "KOSNER BASKONIA VITORIA-GASTEIZ")
            .then(pl.lit("Baskonia Kosner"))
            .otherwise(pl.col("team_name"))
            .alias("team_name"),

        pl.when(pl.col("player").str.contains(","))
            .then(
                pl.col("player")
                .str.split(", ")
                .list.get(1).str.to_titlecase() + 
                pl.lit(" ") + 
                pl.col("player").str.split(", ").list.get(0).str.to_titlecase()
            )
            .otherwise(pl.col("player").str.to_titlecase())
            .alias("player"),
        
        (
            pl.col("minutes")
            .str.split(":")
            .list.get(0).cast(pl.Int64) +
            pl.when(pl.col("minutes").str.split(":").list.get(1).cast(pl.Int64) >= 30)
                .then(1)
                .otherwise(0)
        ).alias("minutes")
    )
    .filter(
        pl.col("team_name").is_in(
            ["Barça", "Valencia Basket", "Real Madrid", "Baskonia Kosner"]
        )
        & (pl.col("date") >= lunes)
        & (pl.col("date") <= viernes)
    )
    .rename({
        "isLeague": "liga",
        "date": "fecha",
        "id_match": "partido",
        "player_id": "id_jug",
        "is_starter": "titular",
        "is_playing": "juega",
        "team_name": "equipo",
        "minutes": "min",
        "points": "pts",
        "field_goals_made2": "tc2_ano",
        "field_goals_attempted2": "tc2_int",
        "field_goals_made3": "tc3_ano",
        "field_goals_attempted3": "tc3_int",
        "free_throws_made": "tl_ano",
        "free_throws_attempted": "tl_int",
        "offensive_rebounds": "reb_of",
        "defensive_rebounds": "reb_def",
        "total_rebounds": "reb",
        "assistances": "ast",
        "steals": "br",
        "turnovers": "bp",
        "blocks_favour": "tap",
        "blocks_against": "tap_con",
        "fouls_commited": "fa",
        "fouls_received": "fal_rec",
        "valuation": "val",
        "plusminus": "mm"
    })
    .select([
        "liga",
        "fecha",
        "player",
        "equipo",
        "min",
        "pts",
        "reb",
        "ast",
        "br",
        "bp",
        "tap", 
        "fa",
        "mm",
        "val"
    ])
)

data_euroleague

liga,fecha,player,equipo,min,pts,reb,ast,br,bp,tap,fa,mm,val
str,datetime[μs],str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""euroleague""",2025-12-16 19:30:00,"""Trey Lyles""","""Real Madrid""",25,14,7,3,2,2,1,3,2,13
"""euroleague""",2025-12-16 19:30:00,"""Alberto Abalde""","""Real Madrid""",23,7,4,2,0,0,0,4,-1,1
"""euroleague""",2025-12-16 19:30:00,"""Facundo Campazzo""","""Real Madrid""",27,25,0,2,3,3,0,1,9,28
"""euroleague""",2025-12-16 19:30:00,"""Chuma Okeke""","""Real Madrid""",12,7,7,0,0,0,1,1,-12,13
"""euroleague""",2025-12-16 19:30:00,"""Gabriele Procida""","""Real Madrid""",4,0,0,0,0,0,0,0,-5,-1
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""euroleague""",2025-12-19 19:30:00,"""Matteo Spagnolo""","""Baskonia Kosner""",12,0,4,0,0,1,1,2,-6,0
"""euroleague""",2025-12-19 19:30:00,"""Trent Forrest""","""Baskonia Kosner""",36,9,3,8,0,3,0,5,-6,7
"""euroleague""",2025-12-19 19:30:00,"""Gytis Radzevicius""","""Baskonia Kosner""",27,3,3,0,1,1,0,1,13,3


In [13]:
df_partidos = (
    data_euroleague
    .group_by(["player", "equipo"])
    .agg([
        pl.col("min").mean(),
        pl.col("pts").mean(),
        pl.col("reb").mean(),
        pl.col("ast").mean(),
        pl.col("br").mean(),
        pl.col("bp").mean(),
        pl.col("tap").mean(),
        pl.col("fa").mean(),
        pl.col("mm").mean(),
        pl.col("val").mean()
    ])
    .sort("player")
)
df_partidos

player,equipo,min,pts,reb,ast,br,bp,tap,fa,mm,val
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Alberto Abalde""","""Real Madrid""",19.0,5.5,2.0,2.5,0.5,0.0,0.0,2.5,-2.5,2.0
"""Alex Len""","""Real Madrid""",8.0,2.0,2.0,0.0,0.0,0.5,2.0,1.0,-2.0,0.5
"""Andres Feliz""","""Real Madrid""",11.0,3.0,2.0,1.0,0.0,0.0,0.0,2.0,-2.0,2.0
"""Brancou Badio""","""Valencia Basket""",15.5,7.0,2.0,1.0,0.0,2.0,0.0,2.5,-5.0,1.0
"""Braxton Key""","""Valencia Basket""",20.0,3.5,5.5,1.0,2.5,1.0,0.5,3.0,16.0,7.0
…,…,…,…,…,…,…,…,…,…,…,…
"""Walter Tavares""","""Real Madrid""",26.0,9.0,5.5,0.0,0.0,2.0,2.5,1.5,-7.5,14.5
"""Will Clyburn""","""Barça""",23.5,15.5,7.0,1.0,0.0,1.0,0.5,1.0,6.0,22.5
"""Willy Hernangomez""","""Barça""",27.0,12.5,7.0,1.0,1.0,1.0,1.0,3.0,8.5,19.0
